# Overview

Clean messy data loaded from `.xlsm` file and export it as a cleaned `.csv`

***This notebook will not run correctly.***

*It requires the original dataset which is witheld for privacy reasons.*

# Imports and data loading

In [40]:
# Filesystem
from google.colab import drive

# Data manipulation
import pandas as pd
import numpy as np

# Data cleaning
import re

# Analysis
import seaborn as sns
import matplotlib.pyplot as plt

The .xlsm file format is effectively a list of .csv files. The years we are interested in is 2010 to 2020 since they are all pretty similar and will be relatively easy to clean.

In [41]:
# Load xlsm
drive.mount('/content/drive', force_remount=True);
path_to_folder = 'drive/My Drive/Colab Notebooks/Fishing/data/'
xlsm = pd.read_excel(path_to_folder + 'fishing2.xlsm', sheet_name=None);

Mounted at /content/drive


In [42]:
# List is a specific order since some cleaning is index based
years = ['2015', '2014', '2013', '2012', '2011', '2010',
         '2020', '2019', '2018', '2017', '2016', ]

# Combine all years
dfs = [xlsm[year] for year in years]
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True);

In [43]:
# # Show data
pd.set_option("max_columns", None)
# dfs

In [44]:
pd.reset_option("max_columns")

# Cleaning data

There is a lot of work to do with this dataset. There are a lot of `NaN` values and a lot of messy columns. Luckily, for some of the columns like 'Fish Size' they were simply renamed/reloacted from other columns like 'Fish Length'. Unfortunately, a lot of the information from the more clean columns like 'Start Time' and 'End Time' were originally inconsistently formatted text locatted in the 'Narrative' column.

In [45]:
# # See where the nans are
pd.set_option("max_columns", None)
df.isnull().groupby(df['Year']).sum()

,Year,Month,Day,Start Time,End Time,Tally,Crew Members,Location,Air Temp,Weather,Wind Level,Wind Direction,Surface Cond,Trolling Speed,Water Depth,Lure,Method,Setup,Additional Info,Fish Type,Fish Size,Narrative,Time,Water Temp,Waves,Fish Length,Lure Type,Lure Brand,Lure Name/Color,Attractor,Surface Temp
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010,0,0,0,244,244,244,244,0,0,0,0,0,244,0,0,244,0,0,244,0,244,0,0,0,0,0,0,0,0,0,244
2011,0,0,0,333,333,333,333,0,0,0,0,0,333,0,0,333,0,0,333,0,333,0,0,0,0,0,0,0,0,0,333
2012,0,0,0,191,191,191,191,0,0,0,0,0,191,0,0,191,0,0,191,0,191,0,0,0,0,0,0,0,0,0,191
2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,199,0,0,227,235,235,235,235,235,235,235,235,235
2014,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,82,1,1,135,163,163,163,163,163,163,163,163,163
2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85,0,0,148,165,165,165,165,165,165,165,165,165
2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,0,1,117,152,152,152,152,152,152,152,152,152
2017,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,29,1,1,30,54,54,54,54,54,54,54,54,54
2018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,0,0,147,188,188,188,188,188,188,188,188,188


In [46]:
pd.reset_option("max_columns")

In [47]:
# Remove rows with a lot of nans
df.drop([265, 1744], inplace=True)

In [48]:
# Initialize dataframe
df_clean = pd.DataFrame()

## Time based columns

### Starting, ending, and caught times

For 2013 to 2020 the start and stop times have their own column, and there are no times marked for when a fish is caught. 

For 2010 to 2012 there is only a column for the time a fish is caught. For most of the entries, the start and stop times are discussed in the narrative. However, the formatting is not consistent and some entries do not have a narrative column.

These values are important to get right since our main metric `hook_rate` is dependant on these values being accurate.

In [49]:
# Fix improperly formatted 'Start Time' entry
df.loc[df['Start Time'] == '0:500', 'Start Time'] = '05:00'
df.loc[367:371, ['Start Time', 'End Time']] = df.loc[367:371, ['End Time', 'Start Time']].values

In [50]:
# Convert column to a datetime object
df_clean['date'] = pd.to_datetime(df[['Year','Month','Day']])
date_stamp = df['Year'].astype(str) + \
             '-' + df['Month'].astype(str) + \
             '-' + df['Day'].astype(str)
df_clean['start_time'] = pd.to_datetime(date_stamp + ' ' + df['Start Time'])
df_clean['end_time'] = pd.to_datetime(date_stamp + ' ' + df['End Time'])
df_clean['time'] = pd.to_datetime(date_stamp + ' ' + df['Time'])

#### Regex extraction for start / end times

We can use regex to extract the `start_time` and `end_time` from the 'Narrative' column. We can take advantage of the fact that for the years 2010-2012 the first two sets of numbers are the start and end time for the fishing trip. There are some exceptions to this where the tally comes first, but we can remove the tallies easily since they have a very consistent format: `{caught} for {hooked}`.

In [51]:
# Extract start and stop times from 'Narrative' column

# Select where 'Start Time' doesn't exist (selects years 2010-2012)
temp1 = df.loc[df['Start Time'].isna(), 'Narrative'].copy()

# Removes colons and tally counts
temp2 = temp1.str.replace(':','')
temp3 = temp2.str.replace(r'(\d+) for (\d+)', '', regex=True)

# Extract start and end times
temp4 = temp3.str.extract(r"(\d+)\D+(\d+)")

# Convert times to millitary time (i.e. 14:30)
def fix_time(time):

    if time is np.nan:
        return np.nan

    # Catches hours formatted like '1' or '14'
    if len(time) < 3:
        temp = f"{time :{'0'}>2}"
        time = f"{temp :{'0'}<4}"

    # Catches hours and minutes formated like '830'
    elif len(time) == 3:
        time = '0' + time

    # Insert colon so conversion to datetime works and return
    return time[:2] + ':' + time[2:]
    
start_times = temp4[0].apply(fix_time)
start_times = pd.to_datetime(date_stamp + ' ' + start_times)
end_times = temp4[1].apply(fix_time)
end_times = pd.to_datetime(date_stamp + ' ' + end_times)

# Set start and stop times as properly formatted datetimes
df_clean.loc[df['Start Time'].isna(), 'start_time'] = start_times
df_clean.loc[df['Start Time'].isna(), 'end_time'] = end_times

#### Imputing start / end times

For times that do not appear in `Start Time`, `End Time`, or `Narrative` we can impute them using the non nan `start_time` and `end_time` values.

For days with a single fish caught, the best solution is to calculate the average time, `time_delta_avg`, between `start_time` and `end_time`, then calculate `start_time` as `time` - `time_delta_avg / 2` and `end_time` as `time` + `time_delta_avg / 2`. Applying the same strategy for days with multiple fish caught would result in different values for `start_time` and `end_time`. The best solution is to use the time that the first fish is caught as `start_time` and the time of the last fish as `end_time`. If the time range is smaller than `time_delta_avg`, calculate the difference between the time deltas. Subtract half of the difference from the first time, and add half of the difference to the last.

There are so few entries that require imputation that any biasing is minimal.

In [52]:
# Impute start and end times for 1087 to 1107 (some missing columns from 2010)

# Calculate average time fished in 2010
mask = (df['Year'] == 2010)
time_deltas = df_clean.loc[mask, 'end_time'] - df_clean.loc[mask, 'start_time']
mean_time_delta = time_deltas.mean()

# Use time of earliest and last fish caught as default 'start_time' and 'end_time'
start_times = df_clean.groupby('date').first()['time']
end_times = df_clean.groupby('date').last()['time']

# Calculate time_deltas of new fish
time_deltas = end_times - start_times

# Select where new time_gaps smaller than mean
time_gaps = np.timedelta64(220, 'm') - time_deltas
time_gaps = time_gaps * (time_gaps > np.timedelta64(0, 'm'))

# Impute new start and end times
time_diff = time_gaps / 2
start_times = start_times - time_diff / 2
end_times = end_times + time_diff / 2

# Set new time ranges only where nans are
def impute_times(row):
    if pd.isnull(row['start_time']):
        row['start_time'] = start_times[row['date']]
    if pd.isnull(row['end_time']):
        row['end_time'] = end_times[row['date']]
    return row
df_clean = df_clean.apply(impute_times, axis=1)

### Imputing time

Use `start_time` and `end_time` to impute fish caught `time` using the middle of the range.

In [53]:
# Impute 'time' for rows with 'end_time' and 'start_time'
time_deltas = df_clean['end_time'] - df_clean['start_time'] 
df_clean.loc[df['Time'].isna(), 'time'] = df_clean['start_time'] + time_deltas / 2

### Year, month

In [54]:
# Add year and month for trends between years and within years
df_clean['year'] = df['Year']
df_clean['month'] = df['Month']

### Hours fishing

In [55]:
# Time between start and stop
time_deltas = df_clean['end_time'] - df_clean['start_time']
df_clean['hours_fishing'] = time_deltas.apply(lambda x: x.seconds / (60 * 60))

### Time periods

Whether the fish was caught in the morning, afternoon, evening, or at night. This implementation does not account for daylights saving or the change in time of sunset and sunrise. 

In [56]:
# Get time periods of the day

def time_to_period(row):

    # Morning is between 6am and 12pm
    if row.hour >= 6 and row.hour < 12:
        return 'morning'

    # Afternooon is between 12pm and 6pm
    elif row.hour >= 12 and row.hour < 18:
        return 'afternoon'

    # Evening is between 6pm and 9pm
    elif row.hour >= 18 and row.hour < 21:
        return 'evening'

    # Night is between 9pm and 6am
    elif row.hour >= 21 and row.hour <= 24:
        return 'night'
    elif row.hour >= 0 and row.hour <= 6:
        return 'night'
        
df_clean['time_period'] = df_clean['time'].apply(time_to_period)

### Seasons

Whether the fish was caught in winter, spring, summer, or fall. These categories are a bit arbitrary. 

In [57]:
# Get seasons of the year

def time_to_period(row):

    # Winter is between December and March
    if row.month >= 1 and row.month < 3:
        return 'winter'
    elif row.month == 12:
            return 'winter'

    # Spring is between March and June
    elif row.month >= 3 and row.month < 6:
        return 'spring'

    # Summer is between June and September
    elif row.month >= 6 and row.month < 9:
        return 'summer'

    # Fall is between September and December
    elif row.month >= 9 and row.month < 12:
        return 'fall'
    
df_clean['season'] = df_clean['time'].apply(time_to_period)

## Tally (fish caught and hooked)

The ideal metric would be the number of fish hooked for every day fishing, even if you didn't catch anything. However, in this dataset if no fish are caught then there is no entry. The next best metric is the number of fish hooked per hour spent fishing. This will at least give us a metric for if a day was good or bad for fishing.

In [58]:
# Use extract() to get tallies from narrative
tallies = df.loc[df['Tally'].isna(), 'Narrative'].str.extract('(\d+\s?for\s?\d+)')

# Split the tally into caught and hooked
split1 = df.loc[df['Year'] > 2012, 'Tally'].str.split('for', expand=True)
split2 = tallies[0].str.split('for', expand=True)
caught = pd.concat((split1[0], split2[0]))
hooked = pd.concat((split1[1], split2[1]))

# Convert to float to handle NaNs properly
df_clean['caught'] = caught.astype(float)
df_clean['hooked'] = hooked.astype(float)

# Impute hooked and caught to be the number of entries 
# (number of fish caught)
caught = df_clean[df_clean['caught'].isna()].groupby('date').size()
def impute_caught_and_hooked(row):
    if pd.isnull(row['caught']):
        row['caught'] = caught[row['date']]
    if pd.isnull(row['hooked']):
        row['hooked'] = caught[row['date']] 
    return row
df_clean.loc[df_clean['caught'].isna()] = df_clean[df_clean['caught'].isna()].apply(impute_caught_and_hooked, axis=1)

## Air temp

In [59]:
# Convert ambiguous temps to discrete temps
df_clean['air_temp'] = df['Air Temp'].astype(str)
air_temps = df['Air Temp'].astype(str)
temps = {
    "30's": 35,
    "40's": 45,
    "50's": 55,
    "60's": 65,
    "70's": 75,
    "80's": 85
}
for temp in temps:
    df_clean.loc[air_temps.str.contains(temp, case=False), 'air_temp'] = temps[temp]
df_clean['air_temp'] = df_clean['air_temp'].astype(int)

## Weather

In [60]:
# Set weather
df_clean['weather'] = df['Weather']

## Wind

Wind-level is close to but not eactly linear with the actual speed. Each level is about a change in windspeed of between three and five mph.

In [61]:
# Make wind level middle of range
split = df['Wind Level'].str.split('-', expand=True)
df_clean['wind_level'] = (split[0].astype(int) + split[1].astype(int)) / 2

In [62]:
# Set wind direction
df_clean['wind_dir'] = df['Wind Direction']

## Trolling speed

In [63]:
# Convert trolling speed ranges to discrete values
df_clean['trolling_speed']  = df['Trolling Speed']
mask = df['Trolling Speed'].astype(str).str.contains('to', )
split = df.loc[mask, 'Trolling Speed'].astype(str).str.split('to', expand=True)
df_clean.loc[split.index, 'trolling_speed'] = (split[0].astype(float) + split[1].astype(float)) / 2
df_clean['trolling_speed'] = df_clean['trolling_speed'].astype(float)

## Water Depth

In [64]:
# Set water depth
df_clean['water_depth'] = df['Water Depth']

## Fish

In [65]:
# 'Fish Length' is good but 'Fish Size' uses ranges
df_clean['fish_length'] = df['Fish Length']
split = df.loc[~df['Fish Size'].isna(), 'Fish Size'].astype(str).str.split('to', expand=True)
df_clean.loc[split.index, 'fish_length'] = (split[0].astype(float) + split[1].astype(float)) / 2

In [66]:
# Set fish type
df_clean['fish_type'] = df['Fish Type']

## Crew count

The number of crew members on a fishing trip influences a couple of factors. Namely, more crew members means that legally more rods can be put out and more fish are allowed to be kept. In consequence, more crew members can affect the hook-rate by inflating it.

The crew members are split between the 'Crew Members' column and the 'Narrative' column depending on the year. The 'Crew Members' column is easiest to extract `crew_count` from. The narrative is more complicated since it includes extra information, and the formatting is not very consistent. We will need to use regex to extract `crew_count` from some of the 'Narrative' entries.

The fisherman who created this dataset tells me that he is allowed three rods per person, but he has never put more than nine rods out. If you are over a certain age you need to have a fishing license to count towards the higher rod limit, so more people does not always guarentee more rods.

In [67]:
# Count crew in 'Crew Members'

def count_crew_members(entry):
    
    # Always one person (self)
    count = 1 

    # Ignore nans
    if entry is np.nan:
        return np.nan

    # Ignore dogs and self
    ignore_names = ['Ruff', 'ruff', 'Buzz', 'buzz', 'Self', 'self']

    # Count crew
    split = entry.split('/')
    for name in split:
        if name not in ignore_names:
            count += 1
    return count

df_clean['crew_count'] = df['Crew Members'].apply(count_crew_members)

In [68]:
# Count crew for all 'Narrative' idxs except 810-918

# Select properly formatted 'Narrative' entries where 'Crew Members' 
# doesn't exist
nar_df = df[df['Crew Members'].isna()]
temp1 = nar_df['Narrative'].str.extract('(^\D*)\d')[0]
temp2 = temp1.str.rstrip()
temp3 = temp2.str.rstrip(',')
temp4 = temp3.str.split('\\W')

# Non-names and dogs (which, unfortunately, don't count towards the rod limit)
ignore_names = ['Ruff', 'ruff', 'Buzz', 'buzz', 
                'Self', 'self', 'Start', 'start',
                'Big', 'and', 'got', 'Two', 'scoop', 
                'his', 'Double', 'fish', 'the', 'lines', 
                'bbbs', 'big', 'an', 'S', 'H', 'tangled', 
                'El', 'net', '']

# Count crew in 'Narrative' column
def count_crew_narrative(entry):

    # Assume at least 1 person for nans
    if entry is np.nan or (entry[0] == '' and len(entry) == 1):
        return 1
    
    # Count crew (including captain)
    count = 1
    for name in entry:
        if name not in ignore_names:
            count += 1
    return count
    
temp5 = temp4.apply(count_crew_narrative)
df_clean.loc[df['Crew Members'].isna(), 'crew_count'] = temp5

In [69]:
# Count crew for 'Narrative' idxs 810-918

# Each group is a section of the 'Narrative' column where all entries are
# formatted identically (start_idx, end_idx, split chunk containing crew)
groups = [
    (810, 821, 2),
    (822, 837, 1),
    (838, 908, 2),
    (909, 910, 1),
    (911, 914, 2)
]

# Count crew for each different group
for group in groups:
    
    # Select pandas series by group range and split 
    # each entry string into chunks
    split = df.loc[group[0] : group[1], 'Narrative'].str.split(',')

    def split_crew(arr):

        # Select chunk and split into a list of crew
        crew = arr[group[2]]
        crew_split = re.split(' and |-| & ', crew)

        # Count crew (including captain)
        count = 1
        for member in crew_split:
            if not member.strip() ==  'self':
                count += 1
        return count

    df_clean.loc[group[0] : group[1], 'crew_count'] = split.apply(split_crew)

## Weight

Since multiple rows represent a single day, the `hook_rate` will be biased towards days with more rows. Therefore individual datapoints should be weighted by the number of entries for a single day.

In [70]:
# Calculate weights for each row

weights_inv = df_clean.groupby('date').size()
df_clean['weight'] = 1.

def apply_weights(row): 
    row['weight'] = 1 / float(weights_inv[row['date']])
    return row

df_clean = df_clean.apply(apply_weights, axis=1)

## Hook-rate

Since only days where a fish is caught are recorded, we need a metric to determine if a day is good or bad. Hooks-per-hour is probably the closest we can get to that metric

In [71]:
# Calculate hooks per hour
df_clean['hook_rate'] = df_clean['hooked'] / df_clean['hours_fishing']

## Location

Some of the sample sizes for these locations is very small, and some aren't even on the lake! We'll drop some rows and impute small samples as `nan`

In [72]:
# Simplify locations

# Later locations overwrite previous locations, order by priority descending
df_clean['location'] = df['Location'].copy()
locations = [
    "AuTrain",
    "White Rocks",
    "Shot Point",
    "Sand Hole",
    "Upper Harbor",
    "Lower Harbor",
    "Chocolay",
    "Lower Harbor Breakwall",
    "Lower Harbor Lighthouse",
]
for location in locations:
    df_clean.loc[df.Location.str.contains(location, case=False), 'location'] = location

# df_clean.groupby('location').size()

In [73]:
# Seul Choix Pt isn't on lake superior, drop it
drop_index = df_clean[df_clean['location'] == 'Seul Choix Pt'].index
df_clean.drop(drop_index, inplace=True)

# Impute nans for locations with less than 30 fish caught
nan_locs = ['AuTrain', 'Fairport', 'Carp', 'Humps', 'Picnic Rocks', 
            'Thoney Point']
for loc in nan_locs:
    df_clean.loc[df_clean['location'] == loc, 'location'] = np.nan

df_clean.groupby('location').size()

location
Chocolay                   284
Golden Triangle             30
Lower Harbor               420
Lower Harbor Breakwall     162
Lower Harbor Lighthouse    239
Sand Hole                   92
Shot Point                 538
Upper Harbor                72
White Rocks                 65
dtype: int64

# Date index dataframe

For calculations where the weight is required (say the mean or standard deviation) we might want to simplify the math. One way is to re-index by date. We can then drop the weight since each row is a single day.

In [74]:
# Impute trolling_speed to mean
df_clean.loc[df_clean['trolling_speed'].isna(), 'trolling_speed'] = df_clean['trolling_speed'].mean()

# impute row 1938 fish_length to mean
df_clean.loc[1938, 'fish_length'] = df_clean['fish_length'].mean()

In [75]:
# Group dataframe by date

# Reduce objects/categorical by mode
by_date_cat = df_clean.groupby('date').agg(pd.Series.mode)
def reduce_arrays(row):

    # Removes all arrays by calculating mean or selecting middle value
    for idx, entry in enumerate(row):

        # Reduce numpy arrays
        if isinstance(entry, np.ndarray):

            # Make empty lists nan
            if len(row.iloc[idx]) == 0:
                row.iloc[idx] = np.nan
            
            # Select mean if numeric
            elif isinstance(entry, float) or isinstance(entry, int):
                row.iloc[idx] = np.mean(entry)

             # Select middle if not numeric
            else:
                row.iloc[idx] = np.take(entry, entry.size // 2)
                
        # Reduce datetime arrays
        elif isinstance(entry, pd.core.arrays.datetimes.DatetimeArray):
            row.iloc[idx] = entry[ len(entry) // 2]

    return row
by_date_cat = by_date_cat.apply(reduce_arrays)

# Reduce numericals by mean
by_date_num = df_clean.groupby('date').mean()

# Combine numericals and categoricals, order matters,
# by_date_num overrides by_date_cat for certain columns
by_date = by_date_cat
for col in by_date_num:
    by_date[col] = by_date_num[col]
by_date.drop(columns='weight', inplace=True)

In [76]:
# When we grouped some of the locations became sparse, let's drop
# any with less than 10 days fished

nan_locs = ['Golden Triangle']
for loc in nan_locs:
    by_date.loc[by_date['location'] == loc, 'location'] = np.nan

by_date.groupby('location').size()

location
Chocolay                    62
Lower Harbor               103
Lower Harbor Breakwall      34
Lower Harbor Lighthouse     64
Sand Hole                   20
Shot Point                  58
Upper Harbor                16
White Rocks                 25
dtype: int64

# Export data

In [77]:
# Save un-reduced dataframes to csv
df_clean.to_csv(path_to_folder + 'fishing_unmerged.csv', index=False)
by_date.to_csv(path_to_folder + 'fishing_by_date_unmerged.csv')